This program randomly selects a subset of Item7 filings from a directory,
then randomly selects a subset of sentences from those filings
that will be hand-labelled.
Some of those hand-labelled sentences will be used for training and some for testing.


Murat Aydogdu 
May 5, 2018

In [1]:
import os, random, csv, time, nltk
from random import shuffle

In [2]:
# This is the root directory under which TXT directory resides
os.chdir('/Users/murataydogdu/Desktop/TextualAnalysis/Version1_2018June_Programs')
!pwd

/Users/murataydogdu/Desktop/TextualAnalysis/Version1_2018June_Programs


In [3]:
# Obtain the names of all the text files in the directory.
# Normally, there should be nothing else here
txtfiles = []
for file in os.listdir("TXT"):
    if file.endswith(".txt"):
        txtfiles.append(file)

In [4]:
# Randomly select 20% of the filings
trtest_files = random.sample(txtfiles, int(len(txtfiles)*0.20))
print len(trtest_files)#, trtest_files       

805


In [5]:
time.ctime()
print nltk.word_tokenize("I am (not) sure this isn't right.")

['I', 'am', '(', 'not', ')', 'sure', 'this', 'is', "n't", 'right', '.']


In [ ]:
with open('/Users/murataydogdu/Desktop/TextualAnalysis/trtest_10K.csv', 'wt') as f:
    writer = csv.writer(f)
    # The file does not need a header. Its output will be:
    # 5 (null sentiment code), trfile, sent_cnt, sent_ind, word_cnt, full_sent separated by _*_
    os.chdir('/Users/murataydogdu/Desktop/TextualAnalysis/Version1_2018June_Programs/TXT')
    for trfile in trtest_files:
        # read the text file first
        with open(trfile, 'rb') as f:
            reader = csv.reader(f)
            sentences = list(reader)
        # total size of the document: sum of number of words in each line
        doclength = 0
        for sentence in sentences:
            # This is for when the sentence has commas in it. 
            # In that case the sentence will be split.
            full_sent = ",".join([str(i) for i in sentence])
            full_sent = nltk.word_tokenize(full_sent.decode("utf8"))
            # This will count punctuation marks as words.
            #print (full_sent, len(full_sent))
            doclength += len(full_sent)
        print trfile, doclength   
        sent_cnt = len(sentences) # Number of sentences in a document
        if doclength >= 2500:     # LM (2011) has 2000 words as treshold.
                                  # I am counting non-word tokens also.
            # Randomly select 5% of sentences and record the indices of these sentences
            sent_indices = random.sample(range(sent_cnt),int(sent_cnt*0.05))

            for sent_ind in sent_indices:
                sent = sentences[sent_ind]
                # Sentences are split if there's a comma: put them back together
                full_sent = ",".join([str(i) for i in sent]) 
                # Number of words in the sentence
                word_cnt = len(nltk.word_tokenize(full_sent.decode("utf8")))
                if (word_cnt <= 50  and word_cnt >= 10):
                    # This is a safety measure: 
                    # a very long sentence can be a sign of incorrect parsing 
                    # (more than one sentence, ...)
                    # Short sentences can be headings, ToC, ...
                    # Note, we are counting tokens not words.
                    #print trfile, sent_cnt, sent_ind, word_cnt, full_sent
                    line = '5 _*_ '+ trfile + ' _*_ ' + "{:04}".format(sent_cnt) + ' _*_ ' + "{:04}".format(sent_ind) + ' _*_ ' + "{:04}".format(word_cnt) + ' _*_ ' + full_sent 
                    print line
                    writer.writerow([line])

In [ ]:
# Reopen the train/test file and shuffle sentences
with open('/Users/murataydogdu/Desktop/TextualAnalysis/Version1_2018June_Programs/trtest_10K.csv', 'rb') as f:
    reader = csv.reader(f)
    sentences = list(reader)
print (len(sentences), type(sentences))   

new_sentences = random.sample(sentences, len(sentences)) 

In [ ]:
# Write the shuffled sentences in a new file
with open('/Users/murataydogdu/Desktop/TextualAnalysis/Version1_2018June_Programs/new_trtest_10K.csv', 'wt') as f:
    writer = csv.writer(f)
    for sentence in new_sentences:
        print sentence
        writer.writerow(sentence)